# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
#WORKING

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.62,78,100,7.34,PN,1699798403
1,1,port-aux-francais,-49.3500,70.2167,5.08,92,100,16.96,TF,1699798405
2,2,biga,40.2281,27.2422,20.11,48,55,1.80,TR,1699798406
3,3,itamaraca,-7.7478,-34.8256,30.03,54,0,4.88,BR,1699798407
4,4,waitangi,-43.9535,-176.5597,12.12,84,58,1.79,NZ,1699798408


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Humidity"],
    alpha=0.4
)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values

#define ideal weather criteria
min_temp = 24
max_temp =30
max_humidity = 50
max_cloudiness = 25
max_wind_speed = 10

#filter the df
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= min_temp) & 
    (city_data_df["Max Temp"] <= max_temp) & 
    (city_data_df["Humidity"] < max_humidity) & 
    (city_data_df["Cloudiness"] < max_cloudiness) & 
    (city_data_df["Wind Speed"] < max_wind_speed)
]

# Drop any rows with null values
ideal_weather_df_cleaned = city_data_df.dropna()

# Display sample data
ideal_weather_df_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.62,78,100,7.34,PN,1699798403
1,1,port-aux-francais,-49.3500,70.2167,5.08,92,100,16.96,TF,1699798405
2,2,biga,40.2281,27.2422,20.11,48,55,1.80,TR,1699798406
3,3,itamaraca,-7.7478,-34.8256,30.03,54,0,4.88,BR,1699798407
4,4,waitangi,-43.9535,-176.5597,12.12,84,58,1.79,NZ,1699798408


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
44,waddan,LY,29.1614,16.1390,35,
57,awsim,EG,30.1230,31.1357,37,
95,illizi,DZ,26.4833,8.4667,18,
221,ghat,LY,24.9647,10.1728,16,
264,umarkot,PK,25.3616,69.7362,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel

radius = 10000
limit = 1

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"

    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},10000",
        "bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
        }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waddan - nearest hotel: No hotel found
awsim - nearest hotel: My home
illizi - nearest hotel: بوناقة
ghat - nearest hotel: فندق تاسيلي
umarkot - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
leribe - nearest hotel: Mountain View Hotel
tsiombe - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
barra da estiva - nearest hotel: Hotel Santa Rita
pokaran - nearest hotel: Pokaran Desert Resort
karachi - nearest hotel: Quetta Agha Hotel
awjilah - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
ajdabiya - nearest hotel: فندق المدينة
calvinia - nearest hotel: No hotel found
douz - nearest hotel: Louage station Douz


,City,Country,Lat,Lng,Humidity,Hotel Name
44,waddan,LY,29.1614,16.1390,35,No hotel found
57,awsim,EG,30.1230,31.1357,37,My home
95,illizi,DZ,26.4833,8.4667,18,بوناقة
221,ghat,LY,24.9647,10.1728,16,فندق تاسيلي
264,umarkot,PK,25.3616,69.7362,16,No hotel found
301,metlili chaamba,DZ,32.2667,3.6333,13,No hotel found
323,leribe,LS,-28.8718,28.0450,11,Mountain View Hotel
327,tsiombe,MG,-25.3000,45.4833,42,No hotel found
349,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
352,barra da estiva,BR,-13.6261,-41.3269,30,Hotel Santa Rita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600, 
    size='Humidity', 
    scale=2, 
    color='City', 
    alpha=.4,
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity']  # Add 'Hotel Name' and 'Country' to hover tooltip
)

# Display the map
city_hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)